# RAG Q&A Agent with LangGraph

This notebook demonstrates a Retrieval-Augmented Generation (RAG) question-answering agent built with LangGraph.

## Architecture

The agent uses a 4-node workflow:

```
User Question → [PLAN] → [RETRIEVE] → [ANSWER] → [REFLECT] → Response
                   ↓          ↓           ↓          ↓
                State      State       State      State
```

- **PLAN**: Determines if retrieval is needed
- **RETRIEVE**: Fetches relevant chunks from ChromaDB vector store
- **ANSWER**: Generates response using LLM with retrieved context
- **REFLECT**: Evaluates answer quality and confidence

## Section 1: Setup and Imports

In [18]:
# Install dependencies (if needed)
# !pip install langgraph langchain chromadb sentence-transformers PyPDF2 python-dotenv openai

import os
import sys
from dotenv import load_dotenv

# Add project root to path
project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("✅ Imports configured")
print(f"Project root: {project_root}")

✅ Imports configured
Project root: /var/www/Q-A-AI-Assistant


In [19]:
# Load environment variables
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")

if HF_TOKEN:
    print("✅ HF_TOKEN loaded successfully")
else:
    print("⚠️ WARNING: HF_TOKEN not found in environment variables")
    print("Please create a .env file with: HF_TOKEN=your_token_here")

✅ HF_TOKEN loaded successfully


In [20]:
# Import agent components
from agents.langgraph_agent import run_workflow, print_result
from agents.vector_store import ChromaVectorStore

print("✅ All imports successful")

✅ All imports successful


## Section 2: Initialize Vector Store

We use ChromaDB for persistent vector storage. This provides:
- Persistent embeddings (survives restarts)
- Metadata support (source file, page numbers)
- Semantic similarity search

**Why ChromaDB over FAISS?**
- ChromaDB offers persistence out of the box
- Better metadata filtering capabilities
- Simpler API for production use

In [21]:
# Initialize ChromaDB vector store
vector_store = ChromaVectorStore(data_folder="data")

# Get statistics
stats = vector_store.get_stats()

print("\n📊 Vector Store Statistics:")
print(f"  Total indexed documents: {stats['total_documents']}")
print(f"  Collection name: {stats['collection_name']}")
print(f"  Storage location: {stats['persist_directory']}")

🔧 Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
🔧 Initializing ChromaDB at: ./chroma_db
✅ Loaded existing collection 'rag_documents' with 2 documents

📊 Vector Store Statistics:
  Total indexed documents: 2
  Collection name: rag_documents
  Storage location: ./chroma_db


In [22]:
# Test retrieval - show what's in the knowledge base
if stats['total_documents'] > 0:
    test_query = "What is the task about?"
    docs, metas, distances = vector_store.query(test_query, top_k=2)
    
    print(f"\n🔍 Sample retrieval for: '{test_query}'\n")
    for i, (doc, meta, dist) in enumerate(zip(docs, metas, distances), 1):
        print(f"Result {i}:")
        print(f"  Source: {meta.get('source', 'unknown')}")
        print(f"  Page: {meta.get('page', '?')}")
        print(f"  Distance: {dist:.4f}")
        print(f"  Text preview: {doc[:150]}...\n")
else:
    print("\n⚠️ No documents in vector store. Please add PDFs to the 'data/' folder.")


🔍 Sample retrieval for: 'What is the task about?'

Result 1:
  Source: task.pdf
  Page: 1
  Distance: 1.4376
  Text preview: TASK 1 Objective: Create a basic AI agent using LangGraph or similar AI Agent framework that can answer questions from a small knowledge base using RA...

Result 2:
  Source: task.pdf
  Page: 2
  Distance: 1.5296
  Text preview:  Use Hugging Face or OpenAI embeddings for vector creation.  Include minimal logging or print statements to show each step ’s output (plan → retriev...



## Section 3: LangGraph Workflow Overview

The workflow is defined in `agents/langgraph_agent.py` with the following structure:

### State Management
An `AgentState` TypedDict flows through all nodes, containing:
- `question`: User input
- `needs_retrieval`: Plan decision
- `retrieved_context`: Retrieved chunks
- `retrieved_metadata`: Source info
- `answer`: LLM response
- `reflection`: Quality evaluation
- `steps_log`: Execution trace

### Workflow Nodes
1. **Plan Node** (`agents/nodes/plan_node.py`): Analyzes if retrieval is needed
2. **Retrieve Node** (`agents/nodes/retrieve_node.py`): Performs semantic search
3. **Answer Node** (`agents/nodes/answer_node.py`): Generates LLM response
4. **Reflect Node** (`agents/nodes/reflect_node.py`): Evaluates quality

### Conditional Logic
If the plan node determines no retrieval is needed (e.g., simple greetings), the workflow skips directly to the answer node.

## Section 4: Test Question 1 - Knowledge Question (With Retrieval)

Testing with a question that requires retrieval from the knowledge base.

In [23]:
# Test Question 1: Knowledge question about LangGraph
question1 = "What is LangGraph and how is it used?"

print("\n" + "="*70)
print("TEST 1: KNOWLEDGE QUESTION (WITH RETRIEVAL)")
print("="*70)

result1 = run_workflow(question1)


TEST 1: KNOWLEDGE QUESTION (WITH RETRIEVAL)

🚀 STARTING RAG WORKFLOW
Question: 'What is LangGraph and how is it used?'

🧠 PLAN NODE
Question: 'What is LangGraph and how is it used?'
Needs Retrieval: True
Reasoning: Knowledge question - retrieval required

🔍 RETRIEVE NODE
Question: 'What is LangGraph and how is it used?'
Needs Retrieval: True
Retrieved 2 chunks:
  1. Source: task.pdf, Page: 1, Distance: 1.5000
     Preview: TASK 1 Objective: Create a basic AI agent using LangGraph or similar AI Agent framework that can ans...
  2. Source: task.pdf, Page: 2, Distance: 1.8921
     Preview:  Use Hugging Face or OpenAI embeddings for vector creation.  Include minimal logging or print stat...

💬 ANSWER NODE
Question: 'What is LangGraph and how is it used?'
Context available: 2454 chars
Calling LLM...
❌ Error calling LLM: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

✅ REFL

In [24]:
# Display detailed results
print_result(result1)


🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

📊 FINAL RESULT

🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

❓ QUESTION:
   What is LangGraph and how is it used?

🧠 PLAN:
   Retrieval needed: True

🔍 RETRIEVED SOURCES:
   1. task.pdf (Page 1)
   2. task.pdf (Page 2)

💬 ANSWER:
   I apologize, but I encountered an error: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

✅ REFLECTION:
   Quality: good
   Confidence: medium

🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷



In [25]:
# Examine individual state components
print("\n📋 Detailed State Inspection:\n")

print("🧠 PLAN Decision:")
print(f"   Needs Retrieval: {result1.get('needs_retrieval')}")

print("\n🔍 RETRIEVED Context (first 300 chars):")
context = result1.get('retrieved_context', '')
print(f"   {context[:300]}..." if len(context) > 300 else f"   {context}")

print("\n💬 ANSWER:")
print(f"   {result1.get('answer', 'N/A')}")

print("\n✅ REFLECTION Metrics:")
reflection = result1.get('reflection', {})
for key, value in reflection.items():
    print(f"   {key}: {value}")


📋 Detailed State Inspection:

🧠 PLAN Decision:
   Needs Retrieval: True

🔍 RETRIEVED Context (first 300 chars):
   [Source: task.pdf, Page: 1]
TASK 1 Objective: Create a basic AI agent using LangGraph or similar AI Agent framework that can answer questions from a small knowledge base using RAG (Retrieval -Augmented Generation) . The goal is to test your understanding of AI agent workflows, RAG pipeline design, a...

💬 ANSWER:
   I apologize, but I encountered an error: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

✅ REFLECTION Metrics:
   answer_length_chars: 203
   answer_length_words: 32
   context_used: True
   has_reflection: False
   confidence: medium
   quality: good
   issues: []


### Analysis of Test 1

The workflow executed all 4 nodes:
1. **PLAN** determined retrieval was needed
2. **RETRIEVE** fetched relevant chunks from ChromaDB
3. **ANSWER** generated a response using the LLM with context
4. **REFLECT** evaluated the answer quality

This demonstrates the full RAG pipeline in action.

## Section 5: Test Question 2 - Outside Knowledge Base

Testing with a question that is NOT in the knowledge base. The system should:
- Still perform retrieval
- Find no relevant context
- Reflect low confidence in the answer

In [26]:
# Test Question 2: Question outside knowledge base
question2 = "What is quantum computing?"

print("\n" + "="*70)
print("TEST 2: QUESTION OUTSIDE KNOWLEDGE BASE")
print("="*70)

result2 = run_workflow(question2)


TEST 2: QUESTION OUTSIDE KNOWLEDGE BASE

🚀 STARTING RAG WORKFLOW
Question: 'What is quantum computing?'

🧠 PLAN NODE
Question: 'What is quantum computing?'
Needs Retrieval: True
Reasoning: Knowledge question - retrieval required

🔍 RETRIEVE NODE
Question: 'What is quantum computing?'
Needs Retrieval: True
Retrieved 2 chunks:
  1. Source: task.pdf, Page: 2, Distance: 1.8067
     Preview:  Use Hugging Face or OpenAI embeddings for vector creation.  Include minimal logging or print stat...
  2. Source: task.pdf, Page: 1, Distance: 1.8395
     Preview: TASK 1 Objective: Create a basic AI agent using LangGraph or similar AI Agent framework that can ans...

💬 ANSWER NODE
Question: 'What is quantum computing?'
Context available: 2454 chars
Calling LLM...
❌ Error calling LLM: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

✅ REFLECT NODE
Question: 'What is quantum computing?'


In [27]:
# Display results
print_result(result2)


🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

📊 FINAL RESULT

🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

❓ QUESTION:
   What is quantum computing?

🧠 PLAN:
   Retrieval needed: True

🔍 RETRIEVED SOURCES:
   1. task.pdf (Page 2)
   2. task.pdf (Page 1)

💬 ANSWER:
   I apologize, but I encountered an error: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

✅ REFLECTION:
   Quality: good
   Confidence: medium

🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷



In [28]:
# Check if reflection detected low confidence
print("\n🔍 Reflection Analysis:")
reflection2 = result2.get('reflection', {})

if reflection2.get('confidence') == 'low':
    print("   ✅ Reflection correctly identified low confidence")
else:
    print(f"   Confidence level: {reflection2.get('confidence', 'unknown')}")

if reflection2.get('issues'):
    print(f"   Issues detected: {reflection2['issues']}")


🔍 Reflection Analysis:
   Confidence level: medium


### Analysis of Test 2

When the question is outside the knowledge base:
- The system still retrieves the most similar chunks
- The LLM should indicate insufficient information
- The reflection node detects this and marks low confidence

This demonstrates the agent's ability to self-evaluate and admit when it doesn't know.

## Section 6: Test Question 3 - Simple Query (No Retrieval)

Testing the conditional workflow where retrieval is skipped.

In [29]:
# Test Question 3: Simple greeting (should skip retrieval)
question3 = "Hello"

print("\n" + "="*70)
print("TEST 3: SIMPLE GREETING (NO RETRIEVAL NEEDED)")
print("="*70)

result3 = run_workflow(question3)


TEST 3: SIMPLE GREETING (NO RETRIEVAL NEEDED)

🚀 STARTING RAG WORKFLOW
Question: 'Hello'

🧠 PLAN NODE
Question: 'Hello'
Needs Retrieval: False
Reasoning: Simple query - no retrieval needed

💬 ANSWER NODE
Question: 'Hello'
Context available: 0 chars
Calling LLM...
❌ Error calling LLM: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

✅ REFLECT NODE
Question: 'Hello'
Answer length: 203 chars

Reflection Results:
  Quality: good
  Confidence: medium
  Has Self-Reflection: False
  Answer Words: 32
  Issues: None


🎯 WORKFLOW COMPLETE

📋 EXECUTION LOG:
  • PLAN: Analyzed question. Retrieval=not required
  • ANSWER: Generated 203 character response using LLM
  • REFLECT: Quality=good, Confidence=medium




In [30]:
# Display results
print_result(result3)


🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

📊 FINAL RESULT

🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

❓ QUESTION:
   Hello

🧠 PLAN:
   Retrieval needed: False

💬 ANSWER:
   I apologize, but I encountered an error: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

✅ REFLECTION:
   Quality: good
   Confidence: medium

🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷



In [31]:
# Verify retrieval was skipped
print("\n🔍 Workflow Path Verification:")

if not result3.get('needs_retrieval'):
    print("   ✅ PLAN correctly determined no retrieval needed")
else:
    print("   ⚠️ PLAN decided retrieval was needed")

if not result3.get('retrieved_context'):
    print("   ✅ RETRIEVE was skipped (no context retrieved)")
else:
    print(f"   Retrieved context length: {len(result3.get('retrieved_context', ''))}")

# Check execution log
print("\n📋 Execution Steps:")
for step in result3.get('steps_log', []):
    print(f"   • {step}")


🔍 Workflow Path Verification:
   ✅ PLAN correctly determined no retrieval needed
   ✅ RETRIEVE was skipped (no context retrieved)

📋 Execution Steps:
   • PLAN: Analyzed question. Retrieval=not required
   • ANSWER: Generated 203 character response using LLM
   • REFLECT: Quality=good, Confidence=medium


### Analysis of Test 3

For simple queries:
- The PLAN node uses heuristics to skip retrieval
- Workflow goes: PLAN → ANSWER → REFLECT (skipping RETRIEVE)
- This saves compute and provides faster responses

This demonstrates the conditional branching capability of LangGraph.

## Section 7: Workflow Execution Log Analysis

Let's examine the step-by-step execution for transparency.

In [32]:
# Compare execution paths of all three tests
print("\n" + "="*70)
print("EXECUTION PATH COMPARISON")
print("="*70)

tests = [
    ("Test 1 (Knowledge Q)", result1),
    ("Test 2 (Outside KB)", result2),
    ("Test 3 (Simple Q)", result3)
]

for test_name, result in tests:
    print(f"\n{test_name}:")
    for step in result.get('steps_log', []):
        print(f"  → {step}")


EXECUTION PATH COMPARISON

Test 1 (Knowledge Q):
  → PLAN: Analyzed question. Retrieval=required
  → RETRIEVE: Found 2 relevant chunks from vector store
  → ANSWER: Generated 203 character response using LLM
  → REFLECT: Quality=good, Confidence=medium

Test 2 (Outside KB):
  → PLAN: Analyzed question. Retrieval=required
  → RETRIEVE: Found 2 relevant chunks from vector store
  → ANSWER: Generated 203 character response using LLM
  → REFLECT: Quality=good, Confidence=medium

Test 3 (Simple Q):
  → PLAN: Analyzed question. Retrieval=not required
  → ANSWER: Generated 203 character response using LLM
  → REFLECT: Quality=good, Confidence=medium


In [33]:
# Timing analysis (if available)
import time

print("\n" + "="*70)
print("PERFORMANCE TEST")
print("="*70)

test_question = "What is the objective of this task?"

start_time = time.time()
perf_result = run_workflow(test_question)
end_time = time.time()

execution_time = end_time - start_time

print(f"\n⏱️ Total execution time: {execution_time:.2f} seconds")
print(f"📊 Steps executed: {len(perf_result.get('steps_log', []))}")


PERFORMANCE TEST

🚀 STARTING RAG WORKFLOW
Question: 'What is the objective of this task?'

🧠 PLAN NODE
Question: 'What is the objective of this task?'
Needs Retrieval: True
Reasoning: Knowledge question - retrieval required

🔍 RETRIEVE NODE
Question: 'What is the objective of this task?'
Needs Retrieval: True
Retrieved 2 chunks:
  1. Source: task.pdf, Page: 1, Distance: 1.5213
     Preview: TASK 1 Objective: Create a basic AI agent using LangGraph or similar AI Agent framework that can ans...
  2. Source: task.pdf, Page: 2, Distance: 1.6277
     Preview:  Use Hugging Face or OpenAI embeddings for vector creation.  Include minimal logging or print stat...

💬 ANSWER NODE
Question: 'What is the objective of this task?'
Context available: 2454 chars
Calling LLM...
❌ Error calling LLM: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

✅ REFLECT NODE
Question: 'What is the obj

## Section 8: Conclusion and Summary

### Agent Capabilities

This RAG Q&A agent demonstrates:

1. **Modular Architecture**: Clear separation of concerns with 4 distinct nodes
2. **State Management**: LangGraph handles state flow between nodes
3. **Conditional Logic**: Intelligent routing based on query type
4. **Retrieval-Augmented Generation**: Combines vector search with LLM generation
5. **Self-Evaluation**: Reflection node assesses answer quality
6. **Transparency**: Full execution logging for debugging

### Known Limitations

1. **Planning Heuristics**: Uses simple keyword matching; could use LLM for better decisions
2. **Chunking Strategy**: Fixed 300-word chunks; could benefit from semantic chunking
3. **Reflection Depth**: Basic quality metrics; could integrate LLM-as-judge or RAGAs
4. **Context Window**: Limited to top-k=3 chunks; might miss relevant information

### Future Improvements

1. **LLM-based Planning**: Use LLM to classify query intent
2. **Adaptive Retrieval**: Dynamically adjust top_k based on query complexity
3. **Re-ranking**: Add reranker between retrieve and answer nodes
4. **Multi-turn Dialogue**: Extend state to support conversation history
5. **Evaluation Metrics**: Integrate RAGAs for automated quality assessment

## Additional: Interactive Testing

Try your own questions below!

In [34]:
# Interactive testing cell - modify the question and run
custom_question = "What framework should I use for this task?"

print(f"\nTesting custom question: '{custom_question}'\n")
custom_result = run_workflow(custom_question)
print_result(custom_result)


Testing custom question: 'What framework should I use for this task?'


🚀 STARTING RAG WORKFLOW
Question: 'What framework should I use for this task?'

🧠 PLAN NODE
Question: 'What framework should I use for this task?'
Needs Retrieval: True
Reasoning: Knowledge question - retrieval required

🔍 RETRIEVE NODE
Question: 'What framework should I use for this task?'
Needs Retrieval: True
Retrieved 2 chunks:
  1. Source: task.pdf, Page: 1, Distance: 1.3448
     Preview: TASK 1 Objective: Create a basic AI agent using LangGraph or similar AI Agent framework that can ans...
  2. Source: task.pdf, Page: 2, Distance: 1.4959
     Preview:  Use Hugging Face or OpenAI embeddings for vector creation.  Include minimal logging or print stat...

💬 ANSWER NODE
Question: 'What framework should I use for this task?'
Context available: 2454 chars
Calling LLM...
❌ Error calling LLM: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to g

---

## Technical Stack Summary

- **Framework**: LangGraph for agent workflow orchestration
- **Vector Database**: ChromaDB for persistent embeddings
- **Embeddings**: sentence-transformers (all-MiniLM-L6-v2)
- **LLM**: HuggingFace Inference API (openai/gpt-oss-20b:nebius)
- **Document Processing**: PyPDF2 for PDF extraction
- **State Management**: TypedDict for type-safe state flow
